# Notes
Some files are missing entries
* Empty rows are being added
* 2003 is an especially bad year for gaps
* Will use 2004-2019 for analysis
* 2004 has many duplicate entries, they appear to follow the few % difference style of 2005

# FIXME
Some files have multiple entries for a single region for a given time stamp
* Currently, I am taking the first entry
* Email NYISO about best idea
* Check to see % difference in the two values

Some files are missing entries
* Still need to fill in values


In [ ]:
import pandas as pd
import numpy as np
from glob import glob
from datetime import datetime, timedelta
import pytz

In [ ]:
# Get UTC from local and DST
def to_utc_dt(date_time, time_zone):
    local = pytz.timezone ('US/Eastern')
    naive = datetime.strptime(date_time, '%m/%d/%Y %H:%M:%S')
    if time_zone == 'EST': # Eastern Standard Time
        is_dst_now = False
    if time_zone == 'EDT': # Eastern Daylight Time
        is_dst_now = True
    local_dt = local.localize(naive, is_dst=is_dst_now)
    utc_dt = local_dt.astimezone(pytz.utc)

    return utc_dt

def get_files(year, month):
    files = glob(f'./nyiso_{year}{month:02}/{year}*.csv')
    files.sort()
    dfs = []
    for f in files:
        #print(f)
        df = nyiso_to_neat_data(f)
        if type(df) == int:
            continue
        else:
            dfs.append(df)
    return dfs

def nyiso_to_neat_data(fname):
    df = pd.read_csv(fname)
    #print(df.head())
    #print(df.tail())
    mapping = {}
    mapping['date_time'] = []
    mapping['old_date_time'] = []
    mapping['time_zone'] = []
    # Get all regions
    # Some data issues lead to regions having multiple entries
    # in first hour
    start_time = -1
    for idx in df.index:
        utc_dt = to_utc_dt(df.loc[idx, 'Time Stamp'], df.loc[idx, 'Time Zone'])
        if start_time == -1:
            start_time = utc_dt
        if utc_dt != start_time:
            break
        if df.loc[idx, 'Name'] not in mapping.keys():
            mapping[df.loc[idx, 'Name']] = []
    #print(mapping)
    # Populate values
    for idx in df.index:
        utc_dt = to_utc_dt(df.loc[idx, 'Time Stamp'], df.loc[idx, 'Time Zone'])
        if utc_dt not in mapping['date_time']:
            mapping['date_time'].append(utc_dt)
            mapping['old_date_time'].append(df.loc[idx, 'Time Stamp'])
            mapping['time_zone'].append(df.loc[idx, 'Time Zone'])
        
        # DUPLICATE ENTRIES
        # Lots of duplicates in 2004 - have not checked them, will likely skip that year
        # The four cases in 2005 have ~ identical values, so avg them
        # The three cases in 2006,2007,2008 have zero entries as one version, so take non-zero one
        if len(mapping['date_time']) == len(mapping[df.loc[idx, 'Name']]):
            print(f"Duplicate region entry for {df.loc[idx, 'Name']} at {df.loc[idx, 'Time Stamp']}")
            # if new values is zero, continue
            if df.loc[idx, 'Integrated Load'] == 0:
                print(f" --- Duplicate with second value as zero {df.loc[idx, 'Name']}, skip")
            # if previous value was zero, replace
            elif mapping[df.loc[idx, 'Name']][-1] == 0:
                print(f" --- Duplicate with initial value as zero, replace with {df.loc[idx, 'Integrated Load']}.")
                mapping[df.loc[idx, 'Name']][-1] = df.loc[idx, 'Integrated Load']
            # else, avg them
            else:
                print(f" --- Duplicate, averaging {mapping[df.loc[idx, 'Name']][-1]} with {df.loc[idx, 'Integrated Load']}.")
                print(f" -------------------------------------------- {mapping[df.loc[idx, 'Name']][-1]} / {df.loc[idx, 'Integrated Load']} = {mapping[df.loc[idx, 'Name']][-1] / df.loc[idx, 'Integrated Load']}")
                mapping[df.loc[idx, 'Name']][-1] = (mapping[df.loc[idx, 'Name']][-1] + df.loc[idx, 'Integrated Load'])/2
                   
        if len(mapping['date_time']) == len(mapping[df.loc[idx, 'Name']]) + 1:
            mapping[df.loc[idx, 'Name']].append(df.loc[idx, 'Integrated Load'])
    #tgt_len = 24
    #for k, v in mapping.items():
    #    if len(v) != tgt_len:
    #        print(f"Length problem with file {fname}")
    #        print(k, len(v))
    #        return -1
    df_new = pd.DataFrame(mapping)
    df_new = sum_ny_state(df_new)
    return df_new

def monthly_file(year, month):
    dfs = get_files(year, month)
    master = dfs[0]
    for i in range(1, len(dfs)):
        master = master.append(dfs[i], ignore_index = True)
    #print(len(master.index))
    master.to_csv(f'./nyiso_{year}{month:02}/demand_summary.csv', index=False)

def sum_ny_state(df):
    regions = df.columns.tolist()
    regions.remove('date_time')
    regions.remove('old_date_time')
    regions.remove('time_zone')
    ny_state = np.zeros(len(df.index))
    for region in regions:
        ny_state += df[region]
    return_df = pd.DataFrame({
        'date_time':df['date_time'],
        'old_date_time':df['old_date_time'],
        'time_zone':df['time_zone'],
        'nyiso demand (MW)':ny_state
    })
    return return_df
    
def annual_file(year):
    for month in range(1, 13):
        df = pd.read_csv(f'./nyiso_{year}{month:02}/demand_summary.csv')
        df['date_time'] = pd.to_datetime(df['date_time'])
        if month == 1:
            master = df
        else:
            master = master.append(df, ignore_index = True)
    
    # Check for missing hours and insert them if needed
    dt_to_add = []
    start = master.iloc[0]['date_time']
    prev = start
    for idx in master.index:
        crnt = master.loc[idx, 'date_time']
        if crnt == start:
            continue
        # Check if sequential
        if crnt != prev + timedelta(hours=1):
            #print(f"Missing data between previous {prev} and idx {idx} dt {crnt}.  Adding an empty row.")
            dt_to_add.append(prev + timedelta(hours=1))
            
            # Add hours until gap is filled
            if crnt != prev + timedelta(hours=2):
                n_hours = 2
                while True:
                    if crnt != prev + timedelta(hours=n_hours):
                        #print(f"Need visual inspection for idx {idx} year {year}")
                        dt_to_add.append(prev + timedelta(hours=n_hours))
                        n_hours += 1
                    else:
                        print(f"Total gap length: {n_hours-1} around {crnt}")
                        break
        prev = crnt
    print(f"dts to add: {dt_to_add}")
    to_add = {}
    for col in master.columns:
        if col == 'date_time':
            to_add[col] = dt_to_add
        else:
            to_add[col] = [np.nan for _ in range(len(dt_to_add))]
    master = master.append(pd.DataFrame(to_add), ignore_index = True)
    master = master.sort_values(['date_time'], ascending = True)
        
    print(f"Length for year {year}: {len(master.index)}")
    master.to_csv(f'./demand_summary_{year}.csv', index=False, na_rep='NA')

# Full range of available data
#for year in range(2002, 2020):

# 2009 onwards has zero gaps (may have multiple entries for a single hour, will check)
for year in range(2004, 2009):
    for month in range(1, 13):
        print(year,month)
        monthly_file(year, month)
    annual_file(year)
    
